In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from directory2 import Directory


#### Defining the directory to analyze

In [2]:
dir = Directory(r'C:\Users\BLUO\AppData\Local\Temp')

In [3]:
# dir.generate_report('./report_split.csv', split_metadata=True)
# dir.generate_report('./report_no_split.csv', split_metadata=False)
# dir.generate_report('./report_filtered.csv', split_metadata=False, filters={'extensions': ['.csv', '.docx', '.pptx', '.xlsx']})
# dir.generate_report('./report_temp.csv', split_metadata=False)

In [5]:
dir.generate_analytics('./analytics_temp.csv')

ValueError: dict contains fields not in fieldnames: 'Size (MB)'

#### Exploring the CSV report

In [ ]:
split_metadata = True
include_text = True
CHAR_LIMIT = 500

data = [file.processor.__dict__ for file in dir.get_files()]

for file in data:
    if include_text:
        file['metadata'] = {k: str(v)[:CHAR_LIMIT] for k, v in file['metadata'].items()}
    elif not include_text:
        for field in ['text', 'docstrings', 'imports', 'words', 'lines']:
            file['metadata'].pop(field, None)
if split_metadata:
    data = pd.json_normalize(data, max_level=1, sep='_')
df = pd.DataFrame(data)
df.columns = df.columns.str.replace('metadata_', '')
df['size'] = df['size'] / 1e6

for boolean in ['is_file', 'is_symlink']:
    df[boolean] = df[boolean].astype(int)

for time in ['modification_time', 'access_time', 'creation_time']:
    df[time] = pd.to_datetime(df[time].round(0), unit='s')

df.replace(np.NaN, 'N/A', inplace=True)
df.columns = df.columns.str.replace('_', ' ')
df.columns = df.columns.str.title()
df.rename(columns={'Size': 'Size (MB)'}, inplace=True)

df

In [ ]:
df.to_csv('./report.csv')

#### Filtering out the data of interest

In [ ]:
fil = pd.DataFrame([{k: v for k, v in file.processor.__dict__.items() if k in ['extension', 'size']} for file in dir.get_files()])
fil['count'] = 1
df = fil.groupby('extension').sum()#.reset_index()
df

#### Exporting the filtered data to CSV

In [ ]:
df.to_csv('./analytics.csv')

#### Visualizing the filtered data

In [ ]:
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)
ax2 = ax.twinx()
ax2.yaxis.get_major_locator().set_params(integer=True)

width = 0.4

(df['size']/1e6).plot(kind='bar', color='#0072b2', ax=ax, width=width, position=1, label='Size (mb)')
df['count'].plot(kind='bar', color='#88ccee', ax=ax2, width=width, position=0, label='Count')

ax.set_ylabel('Size (mb)')
ax2.set_ylabel('Count')
ax.set_xlabel('File type')

plt.legend()
plt.title('File types by Count and Size (mb)')


In [ ]:
import matplotlib.pyplot as plt

labels = df.index
sizes = df['size']

cmap = plt.get_cmap("Pastel2")

plt.title('Pie chart of file types by size')
plt.pie(sizes, labels=labels, colors=cmap(np.arange(11)*4))
plt.show()